### Install and import dependencies

In [0]:
%pip install kaggle
%restart_python

In [0]:
import os
import shutil
from datetime import datetime

#### Setup Kaggle login and import Kaggle

In [0]:
print("Configuring Kaggle credentials...")
try:
    os.environ['KAGGLE_USERNAME'] = dbutils.secrets.get(
        scope="hackathon_secrets", key="kaggle_username")
    os.environ['KAGGLE_KEY'] = dbutils.secrets.get(
        scope="hackathon_secrets", key="kaggle_key")
except Exception as e:
    print(f"Notice: Could not fetch secrets. Checking local environment.")
    if not os.environ.get('KAGGLE_USERNAME') or not os.environ.get('KAGGLE_KEY'):
        raise EnvironmentError(
            "KAGGLE_USERNAME and KAGGLE_KEY must be set via Databricks Secrets or local environment variables.")

from kaggle.api.kaggle_api_extended import KaggleApi  # isort: skip # noqa: E402

### Setup volume

In [0]:
CATALOG = "workspace"
SCHEMA = "car_sales"
VOLUME = "raw_data"
DATASET_NAME = "austinreese/craigslist-carstrucks-data"

TARGET_VOLUME_PATH = f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}/"

try:
    print(f"🔧 Creating schema: {CATALOG}.{SCHEMA}")
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.{SCHEMA}")
    print(f"🔧 Creating volume: {CATALOG}.{SCHEMA}.{VOLUME}")
    spark.sql(f"CREATE VOLUME IF NOT EXISTS {CATALOG}.{SCHEMA}.{VOLUME}")
    print("✅ Volume setup completed successfully!")
except Exception as e:
    print(f"⚠️  Warning: Auto-creation failed. Error: {e}")
    print("💡 You may need to create the catalog/schema/volume manually in the Databricks UI")

### Store Data

In [0]:
print(f"Starting ingestion of {DATASET_NAME} to {TARGET_VOLUME_PATH}...")

api = KaggleApi()
api.authenticate()

print("Downloading from Kaggle...")
print(f"📥 Dataset: {DATASET_NAME}")
print(f"📁 Local download path: {TARGET_VOLUME_PATH}")
api.dataset_download_files(DATASET_NAME, path=TARGET_VOLUME_PATH, unzip=True)

print("📋 Download completed. Files found:")
for filename in os.listdir(TARGET_VOLUME_PATH):
    file_path = os.path.join(TARGET_VOLUME_PATH, filename)
    file_size = os.path.getsize(file_path)
    file_size_mb = file_size / (1024 * 1024)
    file_size_gb = file_size / (1024 * 1024 * 1024)
    size_str = f"{file_size_gb:.2f} GB" if file_size_gb > 1 else f"{file_size_mb:.2f} MB"
    print(f"  📄 {filename}: {size_str}")

total_size = sum(os.path.getsize(os.path.join(TARGET_VOLUME_PATH, f)) for f in os.listdir(TARGET_VOLUME_PATH))
total_size_gb = total_size / (1024 * 1024 * 1024)
print(f"📊 Total download size: {total_size_gb:.2f} GB")

print(f"🏗️  Ensuring Volume exists at {CATALOG}.{SCHEMA}.{VOLUME}...")
print(f"📍 Target volume path: {TARGET_VOLUME_PATH}")

print("Ingestion complete!")

### Load Reparation Costs

In [0]:
csv = """
component,diagnostic,reparation_cost
Brake Pads (Front),squeaking noise when braking,250
Brake Pads (Rear),grinding noise from rear,250
Brake Rotors,steering wheel shakes when braking,450
Brake Caliper,pulling to one side when braking,350
ABS Control Module,ABS light is on,850
Brake Master Cylinder,brake pedal feels spongy,400
Emergency Brake Cable,parking brake not holding,200
Oil Change,routine maintenance due,80
Synthetic Oil Change,routine maintenance synthetic,120
Oil Pan Gasket,oil leak on driveway,450
Valve Cover Gasket,burning oil smell,350
Head Gasket,white smoke from exhaust overheating,2500
Timing Belt,scheduled maintenance ticking noise,900
Timing Chain,rattling noise from engine front,1800
Serpentine Belt,squealing noise on startup,150
Alternator,battery light on dim headlights,550
Starter Motor,car clicks but wont start,450
Battery,car is dead no lights,200
Spark Plugs,engine misfiring rough idle,300
Ignition Coils,check engine light stuttering,600
Fuel Pump,car cranks but wont start,700
Fuel Injector,poor fuel economy gas smell,500
Fuel Filter,hesitation when accelerating,150
Air Filter,dirty engine air filter,50
Radiator,coolant leak overheating,600
Thermostat,engine running cold or hot,250
Water Pump,coolant leak whining noise,550
Coolant Hose,visible leak green fluid,150
Transmission Fluid,transmission slipping,200
Clutch Kit,manual transmission slipping,1200
Transmission Solenoid,stuck in gear check engine light,600
Transmission Rebuild,transmission failure won't move,3500
CV Axle,clicking noise when turning,450
Wheel Bearing,humming noise gets louder with speed,400
Shock Absorbers,bouncy ride clunking over bumps,600
Strut Assembly,rough ride knocking noise,800
Control Arm,clunking noise loose steering,500
Ball Joints,clunking sound uneven tire wear,350
Tie Rod Ends,loose steering wheel,250
Power Steering Pump,whining noise when turning wheel,500
Power Steering Rack,leaking fluid hard to steer,1100
Tire (Economy),worn treads bald tire,120
Tire (Performance),needs new tires,250
Wheel Alignment,car pulls to the left or right,120
Tire Balancing,vibration at highway speeds,80
Bent Rim,vibration thumping sound,200
Muffler,loud exhaust noise,300
Catalytic Converter,check engine light P0420,1500
Exhaust Manifold,loud ticking noise on startup,800
Exhaust Leak,fumes inside cabin,250
Oxygen Sensor,check engine light O2 code,250
MAF Sensor,hesitation stalling,200
Check Engine Light Diag,unknown warning light,150
AC Compressor,ac blowing warm air,900
AC Condenser,ac not working leak found,600
AC Recharge,ac not cold enough,150
Blower Motor,no air coming from vents,300
Heater Core,sweet smell wet floorboard,1000
Window Motor,window stuck wont go up,350
Window Switch,window button not working,150
Door Lock Actuator,door lock stuck,250
Windshield Wipers,streaking on windshield,40
Windshield,cracked windshield stone chip,350
Side Mirror,broken side mirror,300
Headlight Assembly,foggy or broken headlight,250
Tail Light Assembly,cracked tail light,150
Bumper (Front),dent in front bumper,800
Bumper (Rear),rear ended damage,800
Fender,dent in side panel,600
Door Panel,dent in door,500
Hood,hail damage on hood,700
Trunk Lid,trunk wont close properly,400
Paint Touch Up,scratches on paint,200
Full Detail,interior dirty,250
Upholstery Repair,torn seat,300
Headliner,fabric sagging from roof,400
Sunroof Motor,sunroof stuck open,600
Convertible Top,roof leaking or torn,1500
Radio/Head Unit,radio display broken,400
Speakers,sound crackling,200
Backup Camera,camera blue screen,350
Key Fob,remote not working,150
Ignition Switch,key wont turn,300
Battery Terminal,corrosion on battery,50
Fuse,electrical component stopped working,20
Bulb Replacement,light bulb out,30
Wiper Motor,wipers stopped moving,250
Washer Fluid Pump,no water spraying,120
Horn,horn not honking,150
Airbag Sensor,airbag light on,400
Seat Belt,seat belt wont retract,250
Turbocharger,loss of power blue smoke,1800
Diesel Particulate Filter,dpf warning light,2000
AdBlue Heater,def system warning,800
Hybrid Battery,hybrid system warning,2500
EV Charging Port,car wont charge,600
Suspension Air Bag,car sagging on one side,900
Transfer Case,4wd not engaging,1500
Differential,whining noise from rear axle,1200"""

In [0]:
csv_path = os.path.join(TARGET_VOLUME_PATH, "car_repair_costs.csv")
with open(csv_path, "w") as f:
    f.write(csv.strip())
print(f"CSV written to {csv_path}")